In [19]:
from transformers import T5Tokenizer
from tqdm import trange
import os
import random
import torch
from GenMC.utils import read_dataset, save_dataset, read_dataset, set_seed, save_model
from GenMC.model.modeling_genmc import GenMC
from GenMC.run_genmc import get_input_feature
import json
import argparse

# device = torch.device("cuda:0")

ImportError: cannot import name 'compute_rouges' from 'utils' (C:\Users\davse\anaconda3\lib\site-packages\utils\__init__.py)

In [4]:
output_path = "data_with_logits/"
input_path = "GenMC/data/obqa/"

In [5]:
dataset = read_dataset(input_path+"train.jsonl")
len(dataset)

4957

In [13]:
model = GenMC("t5-large", 1, 0.5, 1)
checkpoint = torch.load("GenMC/trained_models/t5-large lr 1e-4 bs 8 epochs 20/pytorch_model.bin")
model.load_state_dict(checkpoint['model_state_dict'])

list

In [ ]:

choice_num = 5
model.eval()
eval_batch_size = 128
step_count = len(dataset) // eval_batch_size
if step_count * eval_batch_size < len(dataset):
    step_count += 1
for i in range(step_count):
    beg_index = i * eval_batch_size
    end_index = min((i + 1) * eval_batch_size, len(dataset))
    batch_example = [example for example in dataset[beg_index:end_index]]

    q_ids, q_mask, qo_ids, qo_mask, clue_ids, answers, output_clue = get_input_feature(
                    batch_example,
                    max_source_length=256,
                    max_len_gen=64,
                    choice_num=choice_num,
                    external_sent_num=None)
    scores, output_sequences, decoder_qo = model(q_ids, q_mask, qo_ids, qo_mask, choice_num, get_decoderqo = True)

    scores = scores.cpu().detach().tolist()
    decoder_qo = decoder_qo.cpu().detach().tolist()
    for idx, j in enumerate(range(beg_index, end_index)):
        dataset[i]["logits"] = scores[idx]
        dataset[i]["H_QC"] = decoder_qo[idx]


In [12]:
save_dataset(output_path+ "train.jsonl",dataset)


dict_keys(['stem', 'choices'])

NameError: name 'scores' is not defined